In [39]:
import ffn
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from get_tickers import get_tickers
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [40]:
start_date = '2024-01-01'
end_date = '2024-01-31'
file_name = f' Monthly_{start_date}_to_{end_date}.csv'

In [41]:
#getting a list of S&P100 stocks
stocks = pd.DataFrame(get_tickers())
tickers = stocks["Symbol"]


In [4]:
#Iterate through the sp100 list to see if there are any tickers that have "issues"
'''
for x in tickers:
    try:
        data = ffn.get(x, start = "2020-6-01")
    except Exception as e:
        print(f'There is a problem with {x} : {e}')
'''


In [42]:
#using the ffn library to pull data
data = ffn.get(tickers, start = start_date, end = end_date)

In [43]:
#take a look at the data pull
data.tail()
#len(data)

,aapl,abbv,abt,acn,adbe,aig,amd,amgn,amt,amzn,...,txn,unh,unp,ups,usb,v,vz,wfc,wmt,xom
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-24,193.764725,159.942307,109.127357,364.389343,606.479980,68.085861,178.289993,298.911194,192.005859,156.869995,...,166.447403,507.256470,238.450424,151.213562,41.027092,270.101898,39.353748,48.493439,52.983467,97.105995
2024-01-25,193.435974,160.828613,110.782455,367.392212,622.580017,67.908653,180.330002,303.568542,195.372498,157.750000,...,163.853836,487.657288,237.702606,154.236298,41.345802,271.056427,40.316620,48.875584,53.755936,99.572647
2024-01-26,191.692596,160.117599,110.348976,366.532837,613.929993,68.450111,177.250000,305.045959,192.523041,159.119995,...,160.594772,497.343231,236.541489,153.840332,41.181618,266.413025,40.421486,49.306721,54.228004,100.420860
2024-01-29,191.005203,159.640381,111.767616,368.784973,630.229980,68.292595,177.830002,306.689758,194.299072,161.259995,...,162.503235,498.667664,241.244949,152.604202,41.481014,272.100433,40.087814,49.336113,54.482185,100.547607
2024-01-30,187.329147,160.624069,111.797180,366.760010,627.960022,69.286896,172.059998,307.834503,190.834839,159.000000,...,159.849762,497.748413,242.750458,140.088379,41.857677,275.570557,40.488220,50.159199,54.663750,102.224541


In [44]:
#calculate the daily and annual returns of the initial data pull
returns_daily = data.to_log_returns().dropna()
returns_annual = returns_daily.mean()*250
#return the length of the daily and annual returns dataframes for verification
len(returns_daily), len(returns_annual)

(19, 100)

In [ ]:
returns_daily = data.to_log_returns().dropna()  # Daily log returns

# Calculate the monthly return for the given month
monthly_return = (returns_daily + 1).prod() - 1  # Compound all daily returns in the month

# Return the value of the monthly return
#monthly_return_value = monthly_return.iloc[0]  # Assuming you only need the scalar value
print("Monthly Returns:", monthly_return)

In [46]:
#sort by annual returns in decreasing from highest annual return to lowest.
sort = pd.DataFrame(returns_annual.sort_values(ascending=False))
print(sort)

             0
nvda  3.484816
amd   2.847309
nflx  2.414171
ibm   1.990067
meta  1.899184
...        ...
intc -1.416944
amt  -1.478113
mmm  -1.825524
ba   -2.999490
tsla -3.417937

[100 rows x 1 columns]


In [47]:
#create a list of top 20 tickers by annual return
top_20=sort[0:20].index
print(top_20)

Index(['nvda', 'amd', 'nflx', 'ibm', 'meta', 'crm', 'avgo', 'vz', 'msft',
       'orcl', 'googl', 'lly', 'axp', 'adbe', 'bk', 'cost', 'c', 'intu', 'mrk',
       'acn'],
      dtype='object')


In [ ]:
#create a new dataframe that has only the top 20 stocks
top_20_df = data[[column for column in top_20]]
top_20_df.head()

In [19]:
top_20_df.to_csv('top_20 ' +file_name)



In [ ]:
'''
def get_open_prices(csv, start, end):

    data = pd.read_csv(csv)
    tickers = data.columns[1:]
    prices_open = ffn.get(tickers, start = open_on_date, end = close_date )
    transposed = prices_open.T
    transposed.columns = ['Price']
    transposed.index.name = 'Stock'
    print(transposed)
    return transposed

get_open_prices(top_20_csv, start_date, end_date)
'''

In [21]:
#calculate the returns per the top_20 df
top_20_ret_daily = top_20_df.to_log_returns().dropna()
top_20_ret_ann = top_20_ret_daily.mean()*30

In [ ]:
#view the annualized returns of the top 20 S&P100 stocks
top_20_ret_ann

In [23]:
#calculate the daily and annual covariances
top_20_cov_daily = top_20_ret_daily.cov()
top_20_cov_annual = top_20_cov_daily*250 #250 trading days in a year.

In [24]:
# create empty lists to store returns, volatility, and weights of possible portfolios
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

In [25]:
####################################################
# set the number of portfolio combinations with    #
# the number of assets and portfolios as variables #
####################################################
num_assets = len(top_20)
num_portfolios = 50000

In [26]:
#set random seed
np.random.seed(42)

In [27]:
# populate the empty lists with each portfolios returns,risk and weights
for single_portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    returns = np.dot(weights, top_20_ret_ann)
    volatility = np.sqrt(np.dot(weights.T, np.dot(top_20_cov_annual, weights)))
    sharpe = returns / volatility
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

In [28]:
# a dictionary for Returns and Risk values of each portfolio
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

for counter,symbol in enumerate(top_20):
    portfolio[symbol+' Weight'] = [Weight[counter] for Weight in stock_weights]

In [29]:
#create the final dataframe with X number of portfolios randomized with different weights
sp_top_20 = pd.DataFrame(portfolio)

In [ ]:
sp_top_20.tail()

In [31]:
# get better labels for desired arrangement of columns
column_order = ['Returns', 'Volatility', 'Sharpe Ratio']+ [stock+' Weight' for stock in top_20]

In [32]:
# reorder dataframe columns
final = sp_top_20[column_order]

In [ ]:
# find min Volatility & max sharpe values in the dataframe (df)
min_volatility = final['Volatility'].min()
max_sharpe = final['Sharpe Ratio'].max()

# use the min, max values to locate and create the two special portfolios
sharpe_portfolio = final.loc[final['Sharpe Ratio'] == max_sharpe]
min_variance_port = final.loc[final['Volatility'] == min_volatility]

# plot frontier, max sharpe & min Volatility values with a scatterplot
plt.style.available
plt.style.use('seaborn-v0_8-darkgrid')
final.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio',
                cmap='RdYlGn', edgecolors='black', figsize=(10, 8), grid=True)
plt.scatter(x=sharpe_portfolio['Volatility'], y=sharpe_portfolio['Returns'], c='red', marker='D', s=100)
plt.scatter(x=min_variance_port['Volatility'], y=min_variance_port['Returns'], c='blue', marker='D', s=100 )
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

In [34]:
min_variance_port.T.as_format(".2%").to_csv('min_var '+file_name)

In [35]:
sharpe_portfolio.T.as_format(".2%").to_csv('max_sharpe ' + file_name)